In [1]:
import pandas as pd

In [2]:
import numpy as np

In [3]:
from sklearn.svm import OneClassSVM

In [4]:
import matplotlib.pyplot as plt

In [6]:
from sklearn.preprocessing import StandardScaler

In [7]:
df = pd.read_csv('records_v2.csv')

In [11]:
print(df.columns.tolist())

['id', 'reading', 'reading_time', 'Fuel Volume (L)']


In [12]:
df['Fuel Volume (L)'] = pd.to_numeric(df['Fuel Volume (L)'], errors='coerce')

In [13]:
df['reading_time'] = pd.to_datetime(df['reading_time'], errors='coerce')

In [14]:
df = df.sort_values(['reading_time', 'id']).reset_index(drop=True)